Read dataset

In [174]:
import sys, os
import re
import pandas as pd
import itertools, nltk, string 

def read_file(file):
    f = open(file, 'r')

    pattern_title = '\[t\]'
    pattern_sentence = '(?<=##).+'
    pattern_aspect = '.+(?=##)'
#     pattern_aspect_anomali = '\[(u|p|s|cc|cs)\]'

    review = []
    for a in f:
        if re.search('##', a):
            sentence = re.findall(pattern_sentence, a)[0]
            aspect = re.findall(pattern_aspect, a)
#             anomali = re.findall(pattern_aspect_anomali, a)
#             if anomali:
#                 print(sentence, aspect)
#             else:
            flag = True
            if len(aspect) > 0:
                aspect = aspect[0]
                
#                 a_array = []
#                 tes = []
#                 tes = aspect.split(', ')
#                 for x in tes:
#                     a_array.append(x.split('[')[0])
#                 for a in a_array:
#                     if a not in sentence:
#                         flag = False
            else:
                aspect = ''
            if flag:
                review.append((sentence, aspect))

    df = pd.DataFrame(columns=['review','target'])
    for r in review:
        df = df.append({'review': r[0], 'target': r[1]}, ignore_index=True)
        
    return df
        
positive_lexicon = []
negative_lexicon = []

def read_lexicon():
    global positive_lexicon;
    global negative_lexicon;
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'positive-words.txt'), 'r') as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
         
        positive_lexicon = file.readlines()
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'negative-words.txt'), 'r', encoding = "ISO-8859-1") as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
        
        negative_lexicon = file.readlines()
        
    positive_lexicon = list(map(lambda word: word.rstrip("\n\r"), positive_lexicon))
    negative_lexicon = list(map(lambda word: word.rstrip("\n\r"), negative_lexicon))
    
        
df = read_file('dataset/bing_liu/Canon G3.txt')
read_lexicon()

In [175]:
df.head()

,review,target
0,i recently purchased the canon powershot g3 an...,canon powershot g3[+3]
1,"the camera is very easy to use , in fact on a ...",use[+2]
2,"after i took their picture with their camera ,...",
3,"i just told them , press halfway , wait for th...",
4,they fired away and the picture turned out qui...,picture[+2]


In [176]:
from nltk.parse.corenlp import CoreNLPDependencyParser

parser = CoreNLPDependencyParser()
parse = next(parser.raw_parse("my name is khan"))

In [177]:
import os
os.environ["CORENLP_HOME"] = r'C:\stanford-corenlp-full-2018-10-05'

import corenlp 
client = corenlp.CoreNLPClient()

def preprocess(sentences, chunk = False):
    res = []
    for sentence in sentences:
        text = sentence.lower()
        if chunk:
            try:
                pattern = '[{pos:NN}][{pos:NN}][{pos:NN}] | [{pos:NN}][{pos:NN}]'
                matches = client.tokensregex(sentence, pattern)
                response = matches['sentences'][0]
                len_chunk = response['length']
                if len_chunk > 0:
                    for index in range(0, len_chunk):
                        replacer = response[str(index)]['text']
                        text = text.replace(replacer, '-'.join(replacer.split(' ')))
            except:
                text = text
        res.append(text)
    return res

In [178]:
import requests
import re

def get_tregex(text):
    url = "http://localhost:9001/tregex"
    request_params = {"pattern": "S < (NP $ VP)"}
    r = requests.post(url, data=text, params=request_params, timeout=120)
    print(r)
    try:
        return r.json()['sentences'][0]
    except:
        return []

def sentence_from_tree(s):
    p_wh = r'(?<=WHADVP).*?(?=\))'
    p_wh2 = r'(?<=WHNP).*?(?=\))'
    pattern = r'(?<= )[a-zA-Z].*?(?=\))'
    replaced = s.replace('\r\n', '')
    wh = re.findall(p_wh, replaced)
    wh2 = re.findall(p_wh2, replaced)
    for x in wh:
        replaced = replaced.replace(x, '')
    for x in wh2:
        replaced = replaced.replace(x, '')
    res = ' '.join(re.findall(pattern, replaced))
    return res

def get_clauses(sentences):
    clauses = []
    res = get_tregex(sentences)
    length = len(res)
    for x in range(0, length):
        clauses.append(sentence_from_tree(res[str(x)]['match']))
    if length > 1:
        for x in range(0, length - 1):
            clauses[x] = clauses[x].replace(clauses[x+1], '')
    return clauses

In [179]:
# adding new dependencies for DP 'nmod' and'advmod'
dep_DP = ['amod', 'prep', 'nsubj', 'csubj', 'xsubj', 'dobj', 'iobj']
conj_DP = ['conj']

In [180]:
def rule_1_1(parse, f, o_expanded):
    #Rule 1.1
    for (w1, dep, w2) in list(parse.triples()):
        if(dep in dep_DP):
            # Rule 1.1
            if(w1[0] in o_expanded):
                if w2[1] == 'NN' and w2[0] not in f:
                    return True, w2[0], w1[0]
            elif(w2[0] in o_expanded):
                if w1[1] == 'NN' and w1[0] not in f:
                    return True, w1[0], w2[0]
    return False, None, None

def rule_1_2(parse, f, o_expanded):
    # Rule 1.2
    for (w1, dep, w2) in parse.triples():
        if(dep in dep_DP):
            H = ''
            O = ''
            if w1[0] in o_expanded:
                H = w2[0]
                O = w1
            elif w2[0] in o_expanded:
                H = w1[0]
                O = w2

            if H:
                for (w1, dep, w2) in list(parse.triples()):
                    if w1[0] == H and w2[0] != O[0]:
                        if w2[1] == 'NN' and w2[0] not in f:
                            return True, w2[0], O[0]
                    elif w2[0] == H  and w1[0] != O[0]:
                        if w1[1] == 'NN' and w1[0] not in f:
                            return True, w1[0], O[0]
    return False, None, None

def rule_4_1(parse, f, o_expanded):
    # Rule 4.1
    for (w1, dep, w2) in list(parse.triples()):
        if(dep in conj_DP):
            if w1[0] in o_expanded:
                if w2[1] == 'JJ' and w2[0] not in o_expanded:
                    return True, w2[0]

            elif w2[0] in o_expanded:
                if w1[1] == 'JJ' and w1[0] not in o_expanded:
                    return True, w1[0]

    return False, None

def rule_4_2(parse, f, o_expanded):
    # Rule 4.2
    for (w1, dep, w2) in parse.triples():
        if(dep in dep_DP or dep in conj_DP):
            H = ''
            O = ''
            if w1[0] in o_expanded:
                H = w2[0]
                O = w1
            elif w2[0] in o_expanded:
                H = w1[0]
                O = w2

            if H:
                for (w1, dep, w2) in list(parse.triples()):
                    if w1[0] == H and w2[0] != O[0]:
                        if w2[1] == 'JJ' and w2[0] not in o_expanded:
                            return True, w2[0]

                    elif w2[0] == H  and w1[0] != O[0]:
                        if w1[1] == 'JJ' and w1[0] not in o_expanded:
                            return True, w1[0]
    
    return False, None

def rule_3_1(parse, f):
    #Rule 3.1
    for (w1, dep, w2) in list(parse.triples()):
        if(dep in conj_DP):
            if(w1[0] in f): 
                if w2[1] == 'NN' and w2[0] not in f:
                    return True, w2[0], w1[0]
            elif(w2[0] in f):          
                if w1[1] == 'NN' and w1[0] not in f:
                    return True, w1[0], w2[0]
                
    return False, None, None

def rule_3_2(parse, f):
    # Rule 3.2
    for (w1, dep, w2) in parse.triples():
        if(dep in dep_DP or dep in conj_DP):
            H = ''
            O = ''
            if w1[0] in f:
                H = w2[0]
                O = w1
            elif w2[0] in f:
                H = w1[0]
                O = w2

            if H:
                for (w1, dep, w2) in list(parse.triples()):
                    if w1[0] == H and w2[0] != O[0]:
                        if w2[1] == 'NN' and w2[0] not in f:
                            return True, w2[0], O[0]
                    elif w2[0] == H  and w1[0] != O[0]:
                        if w1[1] == 'NN' and w1[0] not in f:
                            return True, w1[0], O[0]

    return False, None, None

def rule_2_1(parse, f, o_expanded):
    # Rule 2.1
    for (w1, dep, w2) in list(parse.triples()):
        if(dep in dep_DP):
            if w1[0] in f:
                if w2[1] == 'JJ' and w2[0] not in o_expanded:
                    return True, w2[0]

            elif w2[0] in f:
                if w1[1] == 'JJ' and w1[0] not in o_expanded:
                    return True, w1[0]
                
    return False, None

def rule_2_2(parse, f, o_expanded):      
    # Rule 2.2
    for (w1, dep, w2) in parse.triples():
        if(dep in dep_DP):
            H = ''
            O = ''
            if w1[0] in f:
                H = w2[0]
                O = w1
            elif w2[0] in f:
                H = w1[0]
                O = w2

            if H:
                for (w1, dep, w2) in list(parse.triples()):
                    if w1[0] == H and w2[0] != O[0]:
                        if w2[1] == 'JJ' and w2[0] not in o_expanded:    
                            return True, w2[0]
                    elif w2[0] == H  and w1[0] != O[0]:
                        if w1[1] == 'JJ' and w1[0] not in o_expanded:
                            return True, w1[0]

    return False, None

def clause(other_clause, temp, parse, f):
    res = []
    if other_clause and len(temp) == 0:
        for (w1, dep, w2) in list(parse.triples()):
            if w1[1] == 'NN' and w1[0] not in f:
                res.append(w1[0])
    if len(res) > 0:
        return True, res
    return False, None

In [185]:
import pandas as pd
candidate_aspect = []
new_opinion = []
op_set = positive_lexicon + negative_lexicon

def double_propagation(O: op_set, reviews, targets, output_file, save_to_file = False, using_clause = False):
    o_expanded = O
    f = []
    is_stop = False
    flag_cycle = 0
    
    t_a_p = []
    a_p = []
    r_p = []
    
    while (not is_stop):
        f_i = []
        o_i = []
    
        index = 0

        for sent in reviews:
            sentences = []
            if using_clause and flag_cycle == 0:
                clauses = get_clauses(sent)
                length = len(clauses)
                if length == 0:
                    sentences.append(sent)
                else:
                    for x in range(length - 1, -1, -1):
                        sentences.append(clauses[x])
            else:
                sentences.append(sent)
                
            other_clause = False            

            aspect_sentence = []
            if flag_cycle:
                aspect_sentence = t_a_p[index].split('|')
                
            temp = []
            pair = []
#             if sentences:
#                 r = sentences
            for r in sentences:    
                
                try:
                    parse = next(parser.raw_parse(r))
                except:
                    continue
                    
                # Rule 1.1
                is_true, t_a, o = rule_1_1(parse, aspect_sentence, o_expanded)
                if is_true:
                    f_i.append(t_a)
                    temp.append(t_a)
                    pair.append(t_a + '!' + o)
                    
                # Rule 1.2
                is_true, t_a, o = rule_1_2(parse, aspect_sentence, o_expanded)
                if is_true:
                    f_i.append(t_a)
                    temp.append(t_a)
                    pair.append(t_a + '!' + o)
                
                # Rule 4.1
                is_true, t_o = rule_4_1(parse, f, o_expanded)
                if is_true:
                    o_i.append(t_o)
                
                # Rule 4.2
                is_true, t_o = rule_4_2(parse, f, o_expanded)
                if is_true:
                    o_i.append(t_o)

                    
                if using_clause and flag_cycle == 0:
                    flag_o = False
                    for word in r.split(' '):
                        if word in o_expanded:
                            flag_o = True
                            break

                    # clause
                    is_true, t_a = clause(other_clause, temp, parse, f)
                    if is_true:
                        f_i.append(t_a)
                        temp.append(t_a)

                    if flag_o and len(temp) == 0:
                        other_clause = True

                
            if flag_cycle == 0:
                r_p.append(sent)
                a_array = []
                tes = []
                try:
                    tes = targets[index].split(', ')
                except:
                    tes = []
                for x in tes:
                    splitted = x.split('[')
                    if len(splitted) > 1:
                        a_array.append(splitted[0] + '!' + splitted[1][0])
                    else:
                        a_array.append(splitted[0])

                a_p.append('|'.join(a_array))
                t_a_p.append('|'.join(temp))
                pair.append('|'.join(pair))
            else:
                if len(temp) != 0:
                    t_a_p[index] += '|' + '|'.join(temp)
#                     pair[index] += '|' + '|'.join(pair)
            index += 1
            

        #calculate target and opinion expanded
        f = f + f_i 
        o_expanded = o_expanded + o_i

        
        #reread review, and run rule 3.1, 3.2, 2.1, and 2.2
        index = 0
        f_ii = []
        o_ii = []
        
        for sentences in reviews:
            
            aspect_sentence = []
            if flag_cycle:
                aspect_sentence = t_a_p[index].split('|')
                
            temp = []
            pair = []
            if sentences:
                r = sentences
                
                try:
                    parse = next(parser.raw_parse(r))
                except:
                    continue

                flag_o = False
                for word in r.split(' '):
                    if word in o_expanded:
                        flag_o = True
                        break

                # Rule 3.1
                is_true, t_a, o = rule_3_1(parse, aspect_sentence)
                if is_true:
                    f_ii.append(t_a)
                    temp.append(t_a)
                    pair.append(t_a + '!' + o)
                    
                # Rule 3.2
                is_true, t_a, o = rule_3_2(parse, aspect_sentence)
                if is_true:
                    f_ii.append(t_a)
                    temp.append(t_a)
                    pair.append(t_a + '!' + o)
               

                # Rule 2.1
                is_true, t_o = rule_2_1(parse, f, o_expanded)
                if is_true:
                    o_ii.append(t_o)
                    
                # Rule 2.2
                is_true, t_o = rule_2_2(parse, f, o_expanded)
                if is_true:
                    o_ii.append(t_o)

            if len(temp) != 0:
                t_a_p[index] += '|' + '|'.join(temp)

            index += 1
            
        f_i = f_i + f_ii
        o_i = o_i + o_ii
        f = f + f_ii
        o_expanded = o_expanded + o_ii     
        
        flag_cycle = 1
        
        if(len(f_i) == 0 and len(o_i) == 0):
            if save_to_file == True:
                out = pd.DataFrame(r_p)
                out['aspect'] = a_p
                out['prediction'] = t_a_p
                out.to_csv(output_file)
            is_stop = True
        
    return f, o_expanded

In [186]:
dp_aspect, opinion_expand = double_propagation(op_set, preprocess(df['review'], True), df['target'], 'coba.csv', True, False)

In [187]:
def calculate_frequency(aspects):
    aspect_frequency = {}
    
    for aspect in aspects:
        if(aspect in aspect_frequency):
            aspect_frequency[aspect] += 1
        else:
            aspect_frequency[aspect] = 1
            
    return aspect_frequency

k_DP = ['conj', 'compound']
def pruning_based_on_clause(aspect_frequency, reviews, predictions):
    pruning = []
    index = 0
    for review in reviews:
        temp = predictions[index]
        index += 1
        if isinstance(temp, str):
            prediction = temp.split('|')
        else:
            continue
        parse = next(parser.raw_parse(review))
        for (w1, dep, w2) in list(parse.triples()):
            if(w1[0] in aspect_frequency and w2[0] in aspect_frequency):
                if(dep not in k_DP):
                    if(aspect_frequency[w1[0]] > aspect_frequency[w2[0]]):
                        if w2[0] in prediction:
                            pruning.append(w2[0])
                    elif(aspect_frequency[w1[0]] < aspect_frequency[w2[0]]):
                        if w1[0] in prediction:
                            pruning.append(w1[0])
                
    return pruning
def pruning_based_other_products_and_dealers(aspect_frequency, reviews,predictions, window=3):
    pruning = []
    ProductINDI = ["compare to", "compare with", "better than", "worse than"]
    DealerINDI  = ["shop with", "buy from"]
    count = 0
    for review in reviews:
        temp = predictions[count]
        count += 1
        if isinstance(temp, str):
            prediction = temp.split('|')
        else:
            continue
        if any(indication in review for indication in ProductINDI):
            tokens = nltk.word_tokenize(review)
            index = 0
            while index < len(tokens) - 1:
                if tokens[index] + " " + tokens[index + 1] in ProductINDI:
                    index += 2
                    for x in range(window):
                        next_window = index + x + 1;
                        if next_window < len(tokens) and tokens[next_window] in aspect_frequency:
                            if tokens[next_window] in prediction:
                                pruning.append(tokens[next_window])
                else :
                    index += 1
                    
        if any(indication in review for indication in DealerINDI):
            tokens = nltk.word_tokenize(review)
            index = 0
            while index < len(tokens) - 1:
                if tokens[index] + " " + tokens[index + 1] in DealerINDI:
                    index += 2
                    for x in range(window):
                        next_window = index + x + 1;
                        if next_window < len(tokens) and tokens[next_window] in aspect_frequency:
                            if tokens[next_window] in prediction:
                                pruning.append(tokens[next_window])
                else :
                    index += 1 
                        
    return pruning


In [188]:
import pandas as pd

hs = pd.read_csv("coba.csv")

import re
import math
import numpy as np
target = []

for t in hs['aspect']:
    if t is not np.nan:
        for s in t.split('|'):
            target.append(s)
            
tp = calculate_frequency(target)
ap = calculate_frequency(dp_aspect)
pruning_clause = pruning_based_on_clause(ap,hs['0'], hs['prediction'])
prun_dealer_product = pruning_based_other_products_and_dealers(ap, hs['0'], hs['prediction'])

a = pruning_clause + prun_dealer_product

for x in list(dict.fromkeys(dp_aspect)):
    if ap[x] == 1:
        ap[x] -= 1

for freq in list(dict.fromkeys(a)):
    try:
        if ap[freq] > 0:
            ap[freq] -= 1
    except: continue

new_dp_aspect = []
for a in ap:
    if ap[a] > 0:
        new_dp_aspect.append(a)

In [189]:
def calculate_performance():
    atas_recall = 0
    atas_precision = 0
    bawah_precision = 0
    bawah_recall = 0

    new_target = []
    for x in list(dict.fromkeys(target)):
        for y in x.split(' '):
            new_target.append(y)

    new_aspect = []
    for x in list(dict.fromkeys(new_dp_aspect)):
        for y in x.split('-'):
            new_aspect.append(y)

    for x in tp:
        bawah_precision += tp[x]
        for y in x.split(' '):
            flag = False
#             for apt in new_aspect:
            if y in new_aspect:
                atas_precision += tp[x]
                flag = True
                break
#             if flag:
#                 break

    for x in ap:
        bawah_recall += ap[x]
    #     print(x.split('-'))
        for y in x.split('-'):
            flag = False
#             for tgt in new_target:
            if y in new_target:
                atas_recall += ap[x]
                flag = True
                break
#             if flag:
#                 break

    precision = atas_precision/bawah_precision
    recall = atas_recall/bawah_recall
    f1 = (2 * precision * recall) / (precision + recall)
    
    return precision, recall, f1

In [190]:
calculate_performance()

(0.12156862745098039, 0.18036529680365296, 0.14524211962162323)

In [191]:
ap

{'14x': 0,
 '19in': 0,
 '3lb-subnotebook': 0,
 '4mp': 0,
 '4x': 0,
 '4x-zoom': 0,
 'ability': 3,
 'access': 0,
 'action': 0,
 'addition': 0,
 'adjustment': 0,
 'adobe-camera': 0,
 'alsmost': 0,
 'anybody': 0,
 'anyone': 2,
 'anything': 2,
 'aperture-priority': 2,
 'auto-correction': 0,
 'auto-mode': 0,
 'auto-setting': 0,
 'awe': 0,
 'b': 2,
 'balance': 3,
 'bargain': 0,
 'battery': 2,
 'battery-duration': 0,
 'battery-life': 3,
 'bit': 3,
 'board': 0,
 'book': 0,
 'box': 3,
 'buck': 0,
 'button': 0,
 'buy': 0,
 'buyer': 0,
 'camcorder-battery': 0,
 'camera': 79,
 'camera-bag': 0,
 'camera-experience': 0,
 'camera-review': 0,
 'canon': 16,
 'canon-camera': 0,
 'canon-computer-software': 0,
 'canon-g3': 4,
 'canon-g3-anyone': 0,
 'canon-software': 0,
 'capture': 0,
 'card': 6,
 'casing': 0,
 'category': 0,
 'cent': 0,
 'charge': 2,
 'child': 0,
 'choice': 3,
 'close-up': 0,
 'color': 0,
 'color-response-point': 0,
 'coloration': 0,
 'colorimetry': 0,
 'combination': 1,
 'compactflash': 

In [167]:
calculate_performance()

(0.7331670822942643, 0.6573705179282868, 0.6932030094527682)